In [ ]:
import numpy as np
import pandas as pd

from generators import (
    SolarGenerator,
    WindGenerator,
    NuclearGenerator,
    CoalGenerator,
    GasGenerator,
)
from utils import get_demand_curve

In [ ]:
week = 8

## gameplay

In [ ]:
import inspect
from generators import DataGenerator
from utils import get_demand_curve
from icecream import ic
from matplotlib import pyplot as plt


class EnergyMixer:
    def __init__(self, generators, week) -> None:
        """
        Initialise energy mixer
            Parameters:
                generators (dict): Generator classes in energy mix ,
                week(int): week of year in consideration,
        """
        # get demand curve
        self.demand = get_demand_curve(week=week)

        # initialise generators
        self.generators = {
            k: g(time_steps=list(self.demand.keys()), week=week)
            if DataGenerator in inspect.getmro(g)
            else g(time_steps=list(self.demand.keys()))
            for k, g in generators.items()
        }

    def set_installed_capacity(self, installed_capacity) -> None:
        """
        Set installed capacity for each generator
            Parameters:
                installed_capacity (dict): generator name and installed capacity,
        """
        for k, v in installed_capacity.items():
            self.generators[k].installed_capacity = v

    @property
    def min_power_profiles(self) -> dict:
        return {k: g.min_power for k, g in self.generators.items()}

    def calculate_dispatch(self):
        "calculate dispatch and spare capacity of each generator"

        # initially dispatch levels at minimum for each generator
        dispatch_all = grid.min_power_profiles.copy()
        spare_all = {}

        # loop over generation sources in order of preference
        for col in ["nuclear", "gas", "coal", "solar", "wind"]:
            # calculate shortfall between current dispatch and demand
            shortfall = (
                pd.Series(self.demand) - pd.DataFrame(dispatch_all).sum(axis=1)
            ).clip(lower=0)

            # request generator provides its minimum plus the shortfall
            request = (shortfall + pd.Series(self.generators[col].min_power)).to_dict()
            dispatch_all[col], spare_all[col] = self.generators[col].calculate_dispatch(
                request
            )

        return dispatch_all, spare_all


grid = EnergyMixer(
    generators={
        "solar": SolarGenerator,
        "wind": WindGenerator,
        "nuclear": NuclearGenerator,
        "gas": GasGenerator,
        "coal": CoalGenerator,
    },
    week=6,
)

grid.set_installed_capacity(
    installed_capacity={
        "solar": 5_000e6,
        "wind": 20_000e6,
        "nuclear": 30_000e6,
        "gas": 30_000e6,
        "coal": 10_000e6,
    },
)

grid.calculate_dispatch()

## demand

In [ ]:
demand = get_demand_curve(week=week, population=1)
pd.Series(demand).plot()

In [ ]:
t = list(demand.keys())

## generation

In [ ]:
gen = SolarGenerator(time_steps=t, week=week)
gen.installed_capacity = 1000
dispatch_power, spare_power = gen.calculate_dispatch(
    {k: gen.installed_capacity * 2 for k in gen.min_power.keys()}
)
df = pd.concat(
    [
        pd.Series(gen.min_power).rename("min"),
        pd.Series(gen.max_power).rename("max"),
        pd.Series(dispatch_power).rename("dispatch"),
        pd.Series(spare_power).rename("spare"),
    ],
    axis=1,
)
df.plot()
gen.calculate_costs(df.mean(axis=1))

In [ ]:
gen = WindGenerator(time_steps=t, week=week)
gen.installed_capacity = 1000
dispatch_power, spare_power = gen.calculate_dispatch(
    {k: gen.installed_capacity * 2 for k in gen.min_power.keys()}
)
df = pd.concat(
    [
        pd.Series(gen.min_power).rename("min"),
        pd.Series(gen.max_power).rename("max"),
        pd.Series(dispatch_power).rename("dispatch"),
        pd.Series(spare_power).rename("spare"),
    ],
    axis=1,
)
df.plot()
gen.calculate_costs(df.mean(axis=1))

In [ ]:
gen = NuclearGenerator(time_steps=t)
gen.installed_capacity = 1000
dispatch_power, spare_power = gen.calculate_dispatch(
    {k: gen.installed_capacity * 2 for k in gen.min_power.keys()}
)
df = pd.concat(
    [
        pd.Series(gen.min_power).rename("min"),
        pd.Series(gen.max_power).rename("max"),
        pd.Series(dispatch_power).rename("dispatch"),
        pd.Series(spare_power).rename("spare"),
    ],
    axis=1,
)
df.plot()
gen.calculate_costs(df.mean(axis=1))

In [ ]:
gen = CoalGenerator(time_steps=t)
gen.installed_capacity = 1000
dispatch_power, spare_power = gen.calculate_dispatch(
    {k: gen.installed_capacity * 2 for k in gen.min_power.keys()}
)
df = pd.concat(
    [
        pd.Series(gen.min_power).rename("min"),
        pd.Series(gen.max_power).rename("max"),
        pd.Series(dispatch_power).rename("dispatch"),
        pd.Series(spare_power).rename("spare"),
    ],
    axis=1,
)
df.plot()
gen.calculate_costs(df.mean(axis=1))

In [ ]:
gen = GasGenerator(time_steps=t)
gen.installed_capacity = 1000
dispatch_power, spare_power = gen.calculate_dispatch(
    {k: gen.installed_capacity * 2 for k in gen.min_power.keys()}
)
df = pd.concat(
    [
        pd.Series(gen.min_power).rename("min"),
        pd.Series(gen.max_power).rename("max"),
        pd.Series(dispatch_power).rename("dispatch"),
        pd.Series(spare_power).rename("spare"),
    ],
    axis=1,
)
df.plot()
gen.calculate_costs(df.mean(axis=1))